In [1]:
import keras
from keras import callbacks
from keras.datasets import mnist
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

Using TensorFlow backend.


In [3]:
! nvidia-smi

Mon Dec  3 15:01:32 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                  Off |
| N/A   40C    P0    39W / 300W |    365MiB / 16125MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                  Off |
| N/A   40C    P0    38W / 300W |      0MiB / 16128MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
# out = np.load('out.npy')
# c = np.load('c.npy')
# l = np.load('l.npy')
# q = np.load('q.npy')
# a = np.load('a.npy')

In [4]:
RN_input = np.load('RN_input.npy')

In [5]:
RN_input.shape

(9995, 190, 1536)

In [4]:
print (type(out))
print (out.shape)

<class 'numpy.ndarray'>
(2, 190, 1538)


In [5]:
#  def g_theta(self, RN_input, scope='g_theta', reuse = True, phase = True):
#         """
#         Args
#             RN_input: [o_i, o_j, q], shape = [batch_size*190, 136]
#         Returns
#             g_output: shape = [190, batch_size, 256]
#         """
#         input_dim = RN_input.shape[1]
#         g_units = [256,256,256,256]
#         with tf.variable_scope(scope, reuse = reuse) as scope:
#             g_1 = self.batch_norm_relu(RN_input, g_units[0], scope= 'g_1', phase = phase)
#             g_2 = self.batch_norm_relu(g_1, g_units[1], scope='g_2', phase=phase)
#             g_3 = self.batch_norm_relu(g_2, g_units[2], scope='g_3', phase=phase)
#             g_4 = self.batch_norm_relu(g_3, g_units[3], scope='g_4', phase=phase)
#         g_output = tf.reshape(g_4, shape= [-1, self.batch_size, g_units[3]])
#         return g_output

In [6]:
def g_theta(RN_input, batch_size = 1):
    """
    
    """
    input_shape = RN_input.shape[2]
    g_units = [256,256,256,256]
    
    inputs = Input((input_shape,))
    g1 = Dense(g_units[0], activation = 'relu')(inputs)
    g2 = Dense(g_units[1], activation = 'relu')(g1)
    g3 = Dense(g_units[2], activation = 'relu')(g2)
    g4 = Dense(g_units[3], activation = 'relu')(g3)
    g_out = Reshape((-1, batch_size, g_units[3]))(g4)
    g_sum = K.sum(g_out, axis = 0)    
    
    return g_sum

In [7]:
def f_phi(g_sum):
    """
    f_units[2] is set to be 6 as there are only 6 unique answers for 
    task 2.
    """
    f_units = [256,512,6]
    
    f1 = Dense(f_units[0], activation = 'relu')(g_sum)
    f2 = Dense(f_units[1], activation = 'relu')(f1)
    f3 = Dense(f_units[2], activation = 'relu')(f2)
    
    return f3   

In [8]:
def build(RN_input):
    """
    """
    f_input = g_theta(RN_input)
    pred = f_phi(f_input)
    
    return pred


In [9]:
batch_size = 1

input_shape = out.shape[2]
g_units = [256,256,256,256]

inputs = Input((input_shape,))
g1 = Dense(g_units[0], activation = 'relu')(inputs)
g2 = Dense(g_units[1], activation = 'relu')(g1)
g3 = Dense(g_units[2], activation = 'relu')(g2)
g4 = Dense(g_units[3], activation = 'relu')(g3)
g_out = Reshape((-1, batch_size, g_units[3]))(g4)
g_sum = Lambda(lambda x: K.sum(x, axis=0))(g_out)

# g_sum = K.sum(g_out, axis = 0)    

f_units = [256,512,6]

f1 = Dense(f_units[0], activation = 'relu')(g_out)
f2 = Dense(f_units[1], activation = 'relu')(f1)
f3 = Dense(f_units[2], activation = 'relu')(f2)
    
    

In [10]:
model = Model(input = inputs, output = f3)

/home/nvaitc/anaconda3/envs/rn/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1538)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               393984    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 256)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1, 1, 256)         65792     
__________

In [12]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
label = keras.utils.to_categorical(a)

ValueError: invalid literal for int() with base 10: 'garden'

In [ ]:
model.fit()